In [ ]:
import importlib
import pandas as pd
import utils_tf
import scrap_tf
import time
from tqdm import tqdm
tqdm.pandas()  # active la barre de progression pour .progress_apply()

In [2]:
data = pd.read_csv('players_data-2024_2025.csv')

In [3]:
len(data)

2854

In [4]:
data[data["Player"] == "Kylian Mbappé"]

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Att (GK),Thr,Launch%,AvgLen,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
1691,1692,Kylian Mbappé,fr FRA,FW,Real Madrid,es La Liga,25.0,1998.0,34,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
duplicate_count = data["Player"].duplicated().sum()
print(f"Number of duplicate entries in 'Player': {duplicate_count}")

Number of duplicate entries in 'Player': 152


In [6]:
data = data.drop_duplicates(subset=["Player"])

In [7]:
example = data.iloc[0]

In [8]:
example

Rk                   1
Player      Max Aarons
Nation         eng ENG
Pos                 DF
Squad      Bournemouth
              ...     
Stp                NaN
Stp%               NaN
#OPA               NaN
#OPA/90            NaN
AvgDist            NaN
Name: 0, Length: 267, dtype: object

In [ ]:
# clubs = pd.DataFrame(columns=["id", "name"])
clubs = pd.read_csv("clubs.csv")

In [10]:
# name = example["Player"]
# nation = example["Nation"].strip()[1]
# club = example["Squad"]
# if clubs[clubs["name"] == club].empty:
#     club_search = utils.search_club(club)
#     if club_search:
#         club_id = club_search["id"]
#     clubs = pd.concat([clubs, pd.DataFrame([{"id": club_id, "name": club}])], ignore_index=True)    # add club to clubs dataframe
# else:
#     club_id = clubs[clubs["name"] == club]["id"].values[0]

# status, player_data, marketValueHistory = utils.search_player(name, nationality=nation, club=club_id)
# if "error" not in player_data :
#     if len(marketValueHistory) > 0 :
#         current_club = player_data["club"]["id"]
#         last_club_value = marketValueHistory[-1]["clubId"]
#         value_updated = current_club == last_club_value
#         example["value_updated"] = value_updated
#     else :
#         example["value_updated"] = True  # No market value history, assume value is updated
#     example["market_value"] = marketValueHistory[-1]["marketValue"]
#     example["last_evaluation"] = marketValueHistory[-1]["date"]
#     url = utils.build_player_url(player_data["id"], player_data["name"])
#     example["end_contract"] = scrap.extract_end_contract(url)
    

In [ ]:
def get_info(row):
    name = row["Player"]
    nation = row["Nation"].split()[1] if not pd.isna(row["Nation"]) else None
    club = row["Squad"]
    
    if clubs[clubs["name"] == club].empty:
        club_search = utils.search_club(club)
        if club_search:
            club_id = club_search["id"]
            clubs.loc[len(clubs)] = [club_id, club]
        else:
            club_id = None
    else:
        club_id = clubs[clubs["name"] == club]["id"].values[0]

    status, player_data, marketValueHistory = utils.search_player(name, nation, club_id)

    if "error" not in player_data:
        row["id"] = player_data["id"]
        if len(marketValueHistory) > 0:
            current_club = player_data["club"]["id"]
            last_club_value = marketValueHistory[-1]["clubId"]
            value_updated = current_club == last_club_value
            if "marketValue" in marketValueHistory[-1]:
                row["market_value"] = marketValueHistory[-1]["marketValue"]
                row["last_evaluation"] = marketValueHistory[-1]["date"]
        else:
            value_updated = True
        row["value_updated"] = value_updated
        url = utils.build_player_url(player_data["id"], player_data["name"])
        time.sleep(3)
        row["end_contract"] = scrap.extract_end_contract(url)
        print(url)
        print(row["market_value"])
        print(row["end_contract"])
        row["success"] = True
    else:
        row["success"] = False
    
    return row



In [14]:
data = data.progress_apply(get_info, axis=1)

100%|██████████| 2702/2702 [8:38:32<00:00, 11.51s/it]   


In [15]:
clubs

,id,name
0,989,Bournemouth
1,1049,Valencia
2,410,Udinese
3,244,Marseille
4,618,Saint-Étienne
...,...,...
88,677,Ipswich Town
89,6195,Napoli
90,11,Arsenal
91,18,Gladbach


In [17]:
# save clubs to csv
clubs.to_csv("clubs.csv", index=False)
# save data to csv
data.to_csv("player_data_updated.csv", index=False)

In [32]:
data[data["Player"] == "Kylian Mbappé"].end_contract


1691   NaT
Name: end_contract, dtype: datetime64[ns]

In [36]:
len(data[data["end_contract"].isna()])

2421

In [84]:
def get_end_contract(row):
    if pd.isna(row["end_contract"]) and not pd.isna(row["id"]) and not pd.isna(row["Player"]):
        url = utils.build_player_url(row["id"], row["Player"])
        r = scrap.extract_end_contract(url)
        if r is None :
            time.sleep(5)
            r = scrap.extract_end_contract(url)
        row["end_contract"] = r
    return row

In [85]:
data = data.progress_apply(get_end_contract, axis=1)

100%|██████████| 2702/2702 [12:16:21<00:00, 16.35s/it]  


In [68]:
data_mbappé = data[data["Player"] == "Kylian Mbappé"]

In [69]:
data_mbappé = data_mbappé.progress_apply(get_end_contract, axis=1)
data_mbappé.end_contract

100%|██████████| 1/1 [00:03<00:00,  3.81s/it]


1691   2029-06-30
Name: end_contract, dtype: datetime64[ns]

In [88]:
# save data to csv
data.to_csv("player_data_updated.csv", index=False)

In [86]:
print(len(data[data["end_contract"].notna()]))
print(len(data[data["market_value"].notna()]))
print(len(data))

674
2245
2702


In [79]:
print(data[data["Player"] == "Kylian Mbappé"].market_value)
print(data[data["Player"] == "Kylian Mbappé"].end_contract)
print(data[data["Player"] == "Kylian Mbappé"].id)


1691    170000000.0
Name: market_value, dtype: float64
1691   NaT
Name: end_contract, dtype: datetime64[ns]
1691    342229
Name: id, dtype: object
